In [41]:
import pandas as pd 
import numpy as np 
import yfinance as yf
from datetime import date

import plotly.graph_objects as go

In [42]:
small_cap = ['PSCT', 'PSCH', 'PSCF','PSCD', 'PSCC', 'PSCI', 'PSCE', 'PSCM','PSCU','ROOF']
large_cap = ['XLK', 'XLV', 'XLF', 'XLY', 'XLP', 'XLI', 'XLE', 'XLB', 'XLU', 'XLRE']

start_date = date(2020,1,1)
end_date = date(2025,1,1)

data = yf.download(tickers = small_cap + large_cap,start = start_date, end = end_date)['Close']

returns = data.pct_change().dropna()

[*********************100%***********************]  20 of 20 completed
/var/folders/__/wgnmk9j14kl1krf80l2c6bfw0000gn/T/ipykernel_12195/3941965587.py:9: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



In [43]:
class Momentum:

    def __init__(self, data, returns, lookback, rebalance, type = 'Equal'):
        # Inputs
        self.data = data
        self.returns = returns
        self.lookback = lookback
        self.rebalance = rebalance
        self.type = type

        # Stats
        self.weights = pd.Series(0.0, index=self.returns.columns, dtype=float)
        self.portfolio_returns = pd.Series(0.0, index=self.returns.index, dtype=float)

    def run(self):

        for t in range(self.lookback, len(self.returns.index)):
            # Compute portfolio return for the day
            daily_return = (self.weights * self.returns.iloc[t]).sum()
            self.portfolio_returns.iloc[t] = daily_return

            if t % self.rebalance == 0:

                # Get window returns
                small_cap_window_returns = self.returns.iloc[t - self.lookback:t,][small_cap]
                large_cap_window_returns = self.returns.iloc[t - self.lookback:t,][large_cap]

                # Calculate momentum, long and short threshold
                momentum_small_cap = self.signal(small_cap_window_returns)
                momentum_large_cap = self.signal(large_cap_window_returns)

                # Select assets in the top and bottom deciles
                long_assets = momentum_small_cap.nlargest(3).index
                short_assets = momentum_large_cap.nsmallest(3).index

                # Equally Weighted Portfolio
                self.weights = pd.Series(0.0, index=self.returns.columns, dtype=float)
                self.weights[long_assets] = 1 / len(long_assets) if len(long_assets) > 0 else 0
                self.weights[short_assets] = -1 / len(short_assets) if len(short_assets) > 0 else 0

            

        # Clean + Compute Stats
        self._clean_returns()
        self._compute_stats()

    def signal(self, window_returns):
        """Calculate Momentum Signals"""
        if self.type == 'Equal':

            # Calculate momentum (cumulative returns over the lookback period)
            momentum = (1 + window_returns).prod() - 1 / np.sqrt(window_returns.var())
    
            return momentum
        
    def _clean_returns(self):
        """Clean Portfolio Returns"""

        _df = pd.DataFrame(self.portfolio_returns,columns=['daily_return'])
        _df['cumulative_return'] = (1 + _df['daily_return']).cumprod()
        
        self.portfolio_returns = _df

    def _compute_stats(self):
        """Compute Portfolio Stats"""
        
        # Annualized Return
        annual_ret = self.portfolio_returns['daily_return'].mean()*252

        # Volatility
        annual_vol = self.portfolio_returns['daily_return'].std()*np.sqrt(252)

        # Sharpe Ratio
        sharpe_ratio = annual_ret / annual_vol

        self.stats = {'Sharpe Ratio': round(sharpe_ratio,2),
                      'Annual Return': round(annual_ret,2),
                        'Annual Volatility': round(annual_vol,2)}

In [44]:
m = Momentum(data,returns,120,30,'Equal')

In [45]:
m.run()

In [46]:
portfolio_returns = m.portfolio_returns

In [51]:
benchmark = ['SPY','^SP600']
benchmark_df = yf.download(benchmark,start = start_date,end = end_date)['Close']
benchmark_returns = benchmark_df.pct_change().dropna()

benchmark_portfolio_returns = pd.DataFrame(index = benchmark_returns.index[120:])
benchmark_portfolio_returns['daily_return'] = benchmark_returns.iloc[120:,]['^SP600'] - benchmark_returns.iloc[120:,]['SPY']
benchmark_portfolio_returns['cumulative_return'] = (1+ benchmark_portfolio_returns['daily_return']).cumprod()

[*********************100%***********************]  2 of 2 completed


In [56]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x = portfolio_returns[120:].index,
        y = portfolio_returns[120:]['cumulative_return'],
        name = 'Sector Strategy'
    )
)

fig.add_trace(
    go.Scatter(
        x = benchmark_portfolio_returns.index,
        y = benchmark_portfolio_returns['cumulative_return'],
        name = 'Benchmark (SP600 - SPY)'
    )
)


fig.update_layout(title = 'Sector Strategy vs Benchmark')
fig.update_yaxes(title = 'Cumulative return')
fig.show()